In [1]:
!pip install pandas openai streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.4 MB/s eta 0:00:00


In [5]:
from google.colab import files
uploaded = files.upload()


Saving sample.csv to sample.csv


In [8]:
import pandas as pd

df = pd.read_csv("sample.csv", encoding='latin1')

df.columns = df.columns.str.strip()

df = df.rename(columns={
    "text": "question",
    "response_tweet_id": "answer"
})

df = df[["question", "answer"]]
df = df.dropna()

df.head()

,question,answer
0,@AppleSupport causing the reply to be disregar...,119236
2,@76328 I really hope you all change but I'm su...,119238
3,@105836 LiveChat is online at the moment - htt...,119241
4,@VirginTrains see attached error message. I've...,119243
5,"@105836 Have you tried from another device, Mi...",119244


In [9]:
df.to_csv("customer_support_cleaned.csv", index=False)


In [10]:
files.download("customer_support_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
from openai import OpenAI

client = OpenAI(api_key="YOUR_API_KEY")

st.title("💬 Customer Support Chatbot (Future Interns Task 3)")
st.write("Ask your customer support questions!")

data = pd.read_csv("dataset/customer_support_cleaned.csv")

faq_text = ""
for index, row in data.iterrows():
    faq_text += f"Q: {row['question']} \nA: {row['answer']}\n\n"

def get_response(user_query):
    prompt = f"""
You are a customer support assistant.
Use the FAQ dataset to help the user.

FAQ Knowledge Base:
{faq_text}

User question: {user_query}

If answer is not in FAQs, politely reply based on your customer-care knowledge.
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful support assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message["content"]

user_input = st.text_input("Ask something:")

if user_input:
    answer = get_response(user_input)
    st.write("### 🤖 Response:")
    st.write(answer)


Writing app.py
